#### Here is the neural network that I built. It uses the MNIST digit recognizer dataset. 
#### The link to that dataset is "https://www.kaggle.com/competitions/digit-recognizer". 
#### I built a simple neural network here to solve the problem using only NumPy and Pandas. The accuracy is about 93%.

# Importing libraries

In [2]:
import numpy as np
import pandas as pd

## Reading Data

In [3]:
data = pd.read_csv('../input/digit-recognizer/train.csv')
data

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Changing the data into an array so that we can perform numpy operations on it. 

In [4]:
data = np.array(data)

m,n = data.shape
#data is an mxn matrix now
print('m = ',m,' n = ', n)

##shuffling the data
np.random.shuffle(data)

#taking the tranpose
data_train = data[0:m].T
Y_train = data_train[0]
#print(Y_train)
X_train = data_train[1:n]
#print(X_train)

## image normalisation...
X_train = X_train /255

m =  42000  n =  785


# Building the nerual network now

### function to intilise the parameters 

In [5]:
#generating weights and bias matrices for the layers of the nerual network. Random in the beginning... we'll work on them later... :)
def iprams():
    W1 = np.random.rand(10,784) - 0.5
    b1 = np.random.rand(10,1) - 0.5
    W2 = np.random.rand(10,10) - 0.5
    b2 = np.random.rand(10,1) - 0.5
    return W1, b1, W2, b2


### Code for the activation functions and the derivative function

In [6]:
# used RelU as the actication function to remove linearity among the nodes.
def RelU(Z):
    return np.maximum(0,Z)

# used softmax to convert the vector entries into probablities
def Softmax (Z):
    return np.exp(Z)/sum(np.exp(Z))

def deriv(Z):
    return Z > 0
# the deriv function is such because RelU gives x whose derivative will be 1 for x>0 and gives ) in other cases

### Forward Propagation

In [7]:
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = RelU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = Softmax(Z2)
    return Z1, A1, Z2, A2

### Backward Propagation

In [8]:
#one hot encoding Y and passing  it here
def one_hot (Y):
    one_hot_Y = np.zeros((Y.size, 10))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y
    
def back_prop (Z1, A1, Z2, A2, W2, X, Y):
    m = Y.size
    hotY = one_hot(Y)
    dZ2 = A2 - hotY
    dW2 = 1/m * dZ2.dot(A1.T)
    db2 = 1/m * np.sum (dZ2)
    dZ1 = W2.T.dot(dZ2) * deriv(Z1)
    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum (dZ1)
    return dW1, db1, dW2, db2

### Updating Parameters

In [9]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

### Gradient descent

In [10]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy (predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y)/Y.size

In [13]:
def gradient_descent (X, Y, iterations, alpha):
    W1, b1, W2, b2 = iprams()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params (W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        ## basically to see the progress
        if i%1000 == 0:
            print("iteration: ", i)
            print("Accuracy : ", get_accuracy(get_predictions(A2), Y))
    return W1, b1, W2, b2

In [14]:
W1, b1, W2, b2 = gradient_descent (X_train, Y_train, 10000, 0.1 )

iteration:  0
[3 3 3 ... 3 3 3] [7 1 3 ... 8 0 1]
Accuracy :  0.1645
iteration:  1000
[7 1 3 ... 8 0 1] [7 1 3 ... 8 0 1]
Accuracy :  0.8816666666666667
iteration:  2000
[7 1 3 ... 8 0 1] [7 1 3 ... 8 0 1]
Accuracy :  0.9012857142857142
iteration:  3000
[7 1 3 ... 8 0 1] [7 1 3 ... 8 0 1]
Accuracy :  0.9115
iteration:  4000
[7 1 3 ... 8 0 1] [7 1 3 ... 8 0 1]
Accuracy :  0.9187857142857143
iteration:  5000
[7 1 3 ... 8 0 1] [7 1 3 ... 8 0 1]
Accuracy :  0.9239047619047619
iteration:  6000
[7 1 3 ... 8 0 1] [7 1 3 ... 8 0 1]
Accuracy :  0.9276190476190476
iteration:  7000
[7 1 3 ... 8 0 1] [7 1 3 ... 8 0 1]
Accuracy :  0.9305952380952381
iteration:  8000
[7 1 3 ... 8 0 1] [7 1 3 ... 8 0 1]
Accuracy :  0.933047619047619
iteration:  9000
[7 1 3 ... 8 0 1] [7 1 3 ... 8 0 1]
Accuracy :  0.935047619047619
